Trata-se de um modelo estatístico desenvolvido na década de 1960 para determinar a probabilidade de um evento acontecer, fazendo uso de recursos para prever um saída binária, sendo usado no Machine Learning para análise preditiva. Uma das publicações mais importantes sobre o modelo foi realizada em 1958 por David Cox com o artigo “A Análise de Regressão de Sequências Binárias”.

Cox, D.R. The Regression Analysis of Binary Sequences. Journal of the Royal Statistical Society. Series B (Methodological) Vol. 20, No. 2 (1958), pp. 215-242 (28 pages) Published By: Wiley. http://www.jstor.org/stable/2983890, 1958.

Explicação do algoritmo LOR: 

In [ ]:
#instalar biblioteca Orange Canvas
!pip install Orange3

In [ ]:
#importar biblioteca Orange Canvas
import Orange

In [ ]:
#importar dados do disco local
from google.colab import files  
files.upload()

In [ ]:
#instanciar objeto de dados com base no caminho gerado com a importação do arquivo
dados = Orange.data.Table("/content/Simulacao-1.csv")

In [ ]:
#imprimir os primeiros 5 registros
for d in dados[:5]:
  print(d)

In [ ]:
#explorar os metadados
qtde_campos = len(dados.domain.attributes)
qtde_cont = sum(1 for a in dados.domain.attributes if a.is_continuous)
qtde_disc = sum(1 for a in dados.domain.attributes if a.is_discrete)
print("%d metadados: %d continuos, %d discretos" % (qtde_campos, qtde_cont, qtde_disc))
print("Nome dos metadados:", ", ".join(dados.domain.attributes[i].name for i in range(qtde_campos)),)

In [ ]:
#explorar domínios
dados.domain.attributes

In [ ]:
#explorar classe
dados.domain.class_var

In [ ]:
#explorar dados
print("Campos:", ", ".join(c.name for c in dados.domain.attributes))
print("Registros:", len(dados))
print("Valor do registro 1 da coluna 1:", dados[0][0])
print("Valor do registro 2 da coluna 2:", dados[1][1])

In [ ]:
#criar amostra
qtde100 = len(dados)
qtde70 = len(dados) * 70 / 100
qtde30 = len(dados) * 30 / 100
print("Qtde 100%:", qtde100)
print("Qtde  70%:", qtde70)
print("Qtde  30%:", qtde30)
amostra = Orange.data.Table(dados.domain, [d for d in dados if d.row_index < qtde70])
print("Registros:", len(dados))
print("Registros:", len(amostra))

**Hiperparâmetros**

penalty='l2' - processo que altera a resposta do resultado para ser “mais simples”, sendo frequentemente usado para obter resultados para problemas mal colocados ou para evitar overfitting. Podendo ser l1 - alternativa de mínimos quadrados é Lasso (operador de seleção e contração pelo menos absoluto) ou l2 - 


dual=False

tol=0.0001

C=1.0

fit_intercept=True

intercept_scaling=1

class_weight=None

random_state=None

solver='auto'

max_iter=100

multi_class='auto'

verbose=0

n_jobs=1

preprocessors=None - é usado o pré-processamento padrão quando nenhum outro pré-processador é fornecido. Ele os executa na seguinte ordem: remove instâncias com valores de destino desconhecidos; continua variáveis categóricas (com codificação one-hot); remove colunas vazias; imputa valores ausentes com valores médios.

In [ ]:
#Técnica Logistic Regression (LOR)
lor = Orange.classification.LogisticRegressionLearner(
    penalty='l2', 
    dual=False, 
    tol=0.0001, 
    C=1.0, 
    fit_intercept=True, 
    intercept_scaling=1, 
    class_weight=None, 
    random_state=None, 
    solver='auto', 
    max_iter=100, 
    multi_class='auto', 
    verbose=0, 
    n_jobs=1, 
    preprocessors=None)

In [ ]:
#ligar técnica LOR aos dados
dados_lor = lor(dados)

In [ ]:
#treinar e testar técnica LOR com os dados
avalia_lor = Orange.evaluation.CrossValidation(dados, [lor], k=5)

**Validação**

Para validação dos algoritmos de aprendizado de máquina utilizados nas simulações, foram usadas as métricas de classificação e predição e para entende-las segue explicação:

Verdadeiros Positivos (VP) testes que passam como esperado;

Verdadeiros Negativos (VN) testes que falham devido a falhas reais;

Falsos Positivos (FP) testes que passam, mas não deveriam passar; e,

Falsos Negativos (FN) testes que falham devido à inconsistência no teste.

Para avaliar a assertividade e a regularidade as métricas utilizadas foram:

Acurácia A =  (VP+VN) / (VP+VN+FP+FN);

Precisão P = (VP) / (VP + FP);

Revocação R = (VP) / (VP + FN); e,

F1 F = 2 * ((Precisão * Revocação) / (Precisão + Revocação)).

Para avaliar a previsibilidade e a probabilidade também pode se usar:

Característica de Operação do Receptor na sigla em inglês (ROC), métrica para comparar o desempenho dos modelos, representados pela área sob a curva ROC, que é traçada como um diagrama de VP em função da relação FP, que quanto mais perto o valor for de 1, melhor o desempenho.


In [ ]:
#avaliar os indicadores para o LOR
print("Acurácia:  %.3f" % Orange.evaluation.scoring.CA(avalia_lor)[0])
print("Precisão:  %.3f" % Orange.evaluation.scoring.Precision(avalia_lor)[0])
print("Revocação: %.3f" % Orange.evaluation.scoring.Recall(avalia_lor)[0])
print("F1:        %.3f" % Orange.evaluation.scoring.F1(avalia_lor)[0])
print("ROC:       %.3f" % Orange.evaluation.scoring.AUC(avalia_lor)[0])

In [ ]:
#comparar a técnica LOR com outras 2 técnicas
svm = Orange.classification.SVMLearner(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, max_iter=-1, preprocessors=None)
mlp = Orange.classification.NNClassificationLearner(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, preprocessors=None)
dados_svm = svm(dados)
dados_mlp = mlp(dados)
aprendizado = [lor, svm, mlp]
avaliacao = Orange.evaluation.CrossValidation(dados, aprendizado, k=5)

In [ ]:
#imprimir os indicadores para as 3 técnicas
print(" " * 10 + " | ".join("%-4s" % learner.name for learner in aprendizado))
print("Acurácia  %s" % " | ".join("%.2f" % s for s in Orange.evaluation.CA(avaliacao)))
print("Precisão  %s" % " | ".join("%.2f" % s for s in Orange.evaluation.Precision(avaliacao)))
print("Revocação %s" % " | ".join("%.2f" % s for s in Orange.evaluation.Recall(avaliacao)))
print("F1        %s" % " | ".join("%.2f" % s for s in Orange.evaluation.F1(avaliacao)))
print("ROC       %s" % " | ".join("%.2f" % s for s in Orange.evaluation.AUC(avaliacao)))